## Sentiment Analysis - Amazon baby data

In [2]:
import pandas as pd
#products = pd.read_csv(f"D:/Docs/amazon_baby.csv")
products = pd.read_csv(f"D:/SYED/data/amazon_baby.csv")
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183531 entries, 0 to 183530
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   name    183213 non-null  object
 1   review  182702 non-null  object
 2   rating  183531 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 4.2+ MB


## Remove the NAN values in review column

In [3]:
products = products.fillna({'review':''})  # fill in N/A's in the review column
products['review'].isnull().values.any()

False

## Remove punctuation marks

In [4]:
import string
products['review_clean'] = products['review'].apply(lambda x: x.translate(str.maketrans('', '', 
                           string.punctuation)))
products.head(5)

,name,review,rating,review_clean
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3,These flannel wipes are OK but in my opinion n...
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...


### Remove rating of 3

In [5]:
products = products[products['rating'] != 3]
products.head(3)

,name,review,rating,review_clean
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...


### More than 3 is +1 else -1

In [6]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products['review'][27401]

'I love this product.Simple but does the job great.Very easy to attach.I really have nothing bad to say about it.My baby is now protected from the sun.'

## Load the data as test and train
* `train-idx.json` for Training data
* `test-idx.json` for Test data

In [7]:
train_data_indices = pd.read_json(f"D:/repos/CourseraPlus/1_ML_Combined_Courses/Course/Week-2/train-idx.json")
train_set = pd.DataFrame(products, index = train_data_indices[0])
test_data_indices = pd.read_json(f"D:/repos/CourseraPlus/1_ML_Combined_Courses/Course/Week-2/test-idx.json")
test_set = pd.DataFrame(products, index = test_data_indices[0])
print("Train set: " + str(train_set.shape))
print("Test set: " + str(test_set.shape))

Train set: (133416, 5)
Test set: (33336, 5)


### Remove all the NaN before carrying out training

In [8]:
train_set.dropna(subset = ["name"], inplace=True)
train_set.isnull().values.sum()
test_set.dropna(subset = ["name"], inplace=True)
test_set.isnull().values.sum()
print("Train set: " + str(train_set.shape))
print("Test set: " + str(test_set.shape))

Train set: (121011, 5)
Test set: (30209, 5)


## Dictionary of word counts

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern = r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train_set['review_clean'])
test_matrix = vectorizer.transform(test_set['review_clean'])
print("Train matrix: " + str(train_matrix.shape))
print("Test matrix: " + str(test_matrix.shape))

Train matrix: (121011, 113128)
Test matrix: (30209, 113128)


In [10]:
from sklearn.linear_model import LogisticRegression
sentiment_model = LogisticRegression(random_state=0).fit(train_matrix, train_set['sentiment'])
zero_elem = (sentiment_model.coef_ > 0).sum()
print(zero_elem)

83399


c:\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
#predictions = sentiment_model.predict(test_matrix)
final_scores_data = vectorizer.transform(test_set['review_clean'])
final_score = sentiment_model.decision_function(final_scores_data)
print(final_score)

[ 4.00964325  3.83962325  4.1955828  ...  0.06225689 -1.70672874
  3.60665302]


In [12]:
df = pd.DataFrame(final_score, columns = ['Score'])
print(df.shape)
print(df.columns)
print(test_set.shape)
print(test_set.columns)

(30209, 1)
Index(['Score'], dtype='object')
(30209, 5)
Index(['name', 'review', 'rating', 'review_clean', 'sentiment'], dtype='object')


In [13]:
df_test = test_set
df_test.reset_index(drop = True, inplace = True)

In [14]:
#df2 = pd.concat([df, test_set])
df2 = pd.concat([df, test_set], axis = 1)
df_export = df2.sort_values(by='Score', ascending=False)
df_export.head(5)

,Score,name,review,rating,review_clean,sentiment
20952,71.343315,Joovy Scooter Single Stroller Greenie,"***I've posted an UPDATE at the end***First, l...",5.0,Ive posted an UPDATE at the endFirst let me st...,1.0
22392,57.554052,"Zooper 2011 Waltz Standard Stroller, Flax Brown",I did a TON of research before I purchased thi...,5.0,I did a TON of research before I purchased thi...,1.0
28615,53.675168,Ubbi Cloth Diaper Pail Liner,"(updated 3.22.13) After extensive research, tr...",5.0,updated 32213 After extensive research trial a...,1.0
18044,44.201488,"Infantino Wrap and Tie Baby Carrier, Black Blu...",I bought this carrier when my daughter was abo...,5.0,I bought this carrier when my daughter was abo...,1.0
9166,41.352544,"Joovy Zoom 360 Swivel Wheel Jogging Stroller, ...",The joovy zoom 360 was the perfect solution fo...,5.0,The joovy zoom 360 was the perfect solution fo...,1.0


## Export data (Top & Bot 20)

In [15]:
df_top = df_export.head(20)
#df_top.to_csv('top20.csv')
df_bottom = df_export.tail(20)
#df_bottom.to_csv('bottom20.csv')

## Calculate Accuracy

In [17]:
print("Scores dataframe shape: " + str(df.shape))
print("Test data sentiment as reference shape: " + str(test_set.shape))

Scores dataframe shape: (30209, 1)
Test data sentiment as reference shape: (30209, 5)


In [21]:
accuracy_sentiment_model = sentiment_model.score(final_scores_data, test_set['sentiment'].values)
print("Sentiment model accuracy:\n" + str(accuracy_sentiment_model * 100) + " percent")

Sentiment model accuracy:
93.23711476712238 percent


## Special Classifier

In [22]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [23]:
vectorizer_word_subset = CountVectorizer(vocabulary = significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_set['review_clean'])
test_matrix_word_subset = vectorizer_word_subset.transform(test_set['review_clean'])
print("Train matrix special: " + str(train_matrix_word_subset.shape))
print("Test matrix special: " + str(test_matrix_word_subset.shape))

Train matrix special: (121011, 20)
Test matrix special: (30209, 20)


## Simple Log-Reg model

In [24]:
simple_model = LogisticRegression(random_state=0).fit(train_matrix_word_subset, train_set['sentiment'])
zero_elem_simple = (simple_model.coef_ > 0).sum()
print(zero_elem_simple)

10


In [25]:
# Check the coefficients for 20 words used in model
simple_model.coef_

array([[ 1.36211129,  0.92612655,  1.18396582,  0.12316404,  0.4992867 ,
         1.51624378,  1.74488656,  0.47048104,  0.18216587,  0.09178103,
        -1.66839038, -0.18828847, -0.520762  , -2.01335828, -2.35501089,
        -0.63724073, -0.31017645, -0.88476178, -0.33758629, -2.02095452]])